In [1]:
import pandas as pd
import dexpy
from dexpy import factorial

factors = {'Triton  / %':0.01,'DMSO /%':4,'Arginine / mM':150,'Glucose / mM':150,\
           'KPi / mM':150,'KCl / mM':150,'NaCl / mM': 150,'NH4 / mM': 150,'BSA /mg/ml':0.1}

design = factorial.build_factorial(len(factors),2**4)
design.columns = factors

for i,j in zip(design,factors):
    design.loc[:,i].replace({-1:0,1:factors[i]},inplace=True)
design

,Triton / %,DMSO /%,Arginine / mM,Glucose / mM,KPi / mM,KCl / mM,NaCl / mM,NH4 / mM,BSA /mg/ml
0,0.00,0,0,0,0,0,0,0,0.1
1,0.00,0,0,150,0,150,150,150,0.0
2,0.00,0,150,0,150,150,150,0,0.0
3,0.00,0,150,150,150,0,0,150,0.1
4,0.00,4,0,0,150,150,0,150,0.0
5,0.00,4,0,150,150,0,150,0,0.1
6,0.00,4,150,0,0,0,150,150,0.1
7,0.00,4,150,150,0,150,0,0,0.0
8,0.01,0,0,0,150,0,150,150,0.0
9,0.01,0,0,150,150,150,0,0,0.1


In [112]:
StockConcs = {'Triton  / %':10,'DMSO /%':100,'Arginine / mM':500,'Glucose / mM':1000,\
           'KPi / mM':1000,'KCl / mM':1000,'NaCl / mM': 1000,'NH4 / mM': 1000,'BSA /mg/ml':10}

def V1(v2,C1,C2):
    return (C2*v2)/C1


def VolToAdd(RowFromExpDesign, StockConcs,TotalVol_ul):
    # columns in the design and keys in the stock concs need to 
    # be in the same order
    
    VolToAddCols = ['Triton  / ul','DMSO / ul','Arginine / ul','Glucose / ul',\
                    'KPi / ul','KCl / ul','NaCl / ul','NH4 / ul','BSA / ul']
    
    output = pd.DataFrame([[V1(TotalVol_ul, StockConcs[j],i) for i,j in zip(RowFromExpDesign, StockConcs)]],\
                          columns = VolToAddCols)

    VolWater = TotalVol_ul-output.sum().sum()
    output['Water'] = VolWater
    return output



plan = pd.DataFrame([],columns = ['Triton  / ul','DMSO / ul','Arginine / ul','Glucose / ul',\
                    'KPi / ul','KCl / ul','NaCl / ul','NH4 / ul','BSA / ul'])

for i in design.index:
    temp = VolToAdd(design.loc[i,:],StockConcs, 5_000)
    plan=plan.append(temp)

    
plan = plan.reset_index(drop=True)
plan

,Arginine / ul,BSA / ul,DMSO / ul,Glucose / ul,KCl / ul,KPi / ul,NH4 / ul,NaCl / ul,Triton / ul,Water
0,0.0,50.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4950.0
1,0.0,0.0,0.0,750.0,750.0,0.0,750.0,750.0,0.0,2000.0
2,1500.0,0.0,0.0,0.0,750.0,750.0,0.0,750.0,0.0,1250.0
3,1500.0,50.0,0.0,750.0,0.0,750.0,750.0,0.0,0.0,1200.0
4,0.0,0.0,200.0,0.0,750.0,750.0,750.0,0.0,0.0,2550.0
5,0.0,50.0,200.0,750.0,0.0,750.0,0.0,750.0,0.0,2500.0
6,1500.0,50.0,200.0,0.0,0.0,0.0,750.0,750.0,0.0,1750.0
7,1500.0,0.0,200.0,750.0,750.0,0.0,0.0,0.0,0.0,1800.0
8,0.0,0.0,0.0,0.0,0.0,750.0,750.0,750.0,5.0,2745.0
9,0.0,50.0,0.0,750.0,750.0,750.0,0.0,0.0,5.0,2695.0


In [111]:
plan.sum()/1000

Arginine / ul    24.00
BSA / ul          0.80
DMSO / ul         3.20
Glucose / ul     12.00
KCl / ul         12.00
KPi / ul         12.00
NH4 / ul         12.00
NaCl / ul        12.00
Triton  / ul      0.08
Water            71.92
dtype: float64

In [113]:
plan.sort_values('Water')

,Arginine / ul,BSA / ul,DMSO / ul,Glucose / ul,KCl / ul,KPi / ul,NH4 / ul,NaCl / ul,Triton / ul,Water
15,1500.0,50.0,200.0,750.0,750.0,750.0,750.0,750.0,5.0,-505.0
3,1500.0,50.0,0.0,750.0,0.0,750.0,750.0,0.0,0.0,1200.0
2,1500.0,0.0,0.0,0.0,750.0,750.0,0.0,750.0,0.0,1250.0
6,1500.0,50.0,200.0,0.0,0.0,0.0,750.0,750.0,0.0,1750.0
7,1500.0,0.0,200.0,750.0,750.0,0.0,0.0,0.0,0.0,1800.0
10,1500.0,50.0,0.0,0.0,750.0,0.0,750.0,0.0,5.0,1945.0
11,1500.0,0.0,0.0,750.0,0.0,0.0,0.0,750.0,5.0,1995.0
1,0.0,0.0,0.0,750.0,750.0,0.0,750.0,750.0,0.0,2000.0
5,0.0,50.0,200.0,750.0,0.0,750.0,0.0,750.0,0.0,2500.0
14,1500.0,0.0,200.0,0.0,0.0,750.0,0.0,0.0,5.0,2545.0


In [153]:
plan.to_csv('20191109_Exp1_BufferScreenPlan.csv')
design.to_csv('20191109_Exp1_BufferScreenDesign.csv')

In [145]:
import multiprocessing
from multiprocessing import Process


help(Process)

Help on class Process in module multiprocessing.context:

class Process(multiprocessing.process.BaseProcess)
 |  Process(group=None, target=None, name=None, args=(), kwargs={}, *, daemon=None)
 |  
 |  Process objects represent activity that is run in a separate process
 |  
 |  The class is analogous to `threading.Thread`
 |  
 |  Method resolution order:
 |      Process
 |      multiprocessing.process.BaseProcess
 |      builtins.object
 |  
 |  Methods inherited from multiprocessing.process.BaseProcess:
 |  
 |  __init__(self, group=None, target=None, name=None, args=(), kwargs={}, *, daemon=None)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  __repr__(self)
 |      Return repr(self).
 |  
 |  close(self)
 |      Close the Process object.
 |      
 |      This method releases resources held by the Process object.  It is
 |      an error to call this method if the child process is still running.
 |  
 |  is_alive(self)
 |      Return whether process i

In [151]:
import time

def func(n, return_dict):
    return_dict[n] = (10*n)


jobs = []

manager = multiprocessing.Manager()
return_dict = manager.dict()
for i in range(6):
    p = Process(target = func, args = (i, return_dict))
    jobs.append(p)
    p.start()
    

for i in jobs:
    i.join()
return_dict.values()

[0, 10, 30, 20, 40, 50]